# Gendering the talks 

One of our areas of interest are to see if TED talks by men and women are different. If there are differences, we would like to see if we can detect the gender of a speaker based on the words in a TED talk transcript. To pursue these avenues, we begin by "gendering" talks when possible. 

In this notebook, we use the genders of the speakers to "gender" the TED talks themselves. A talk with one speaker inherits the gender of the speaker. For a talk with two speakers, if the genders of the speakers are the same, then we proceed as if the talk had one speaker. For talks with two speakers where the genders of the speakers is not the same, we place these talks to the side. 

In [1]:
import pandas as pd
import csv
import string

In [2]:
# Load the gendered speaker file:
speakers = pd.read_csv("speakers_with_gender.csv")

In [3]:
ted_only = pd.read_csv('../data/Release_v0/TEDonly_final.csv')
ted_plus = pd.read_csv('../data/Release_v0/TEDplus_final.csv')

In [4]:
ted_only.head()

,Unnamed: 0,Unnamed: 0.1,Talk_ID,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4
0,0,0,1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",Al Gore,NaN,NaN,NaN
1,1,1,7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",David Pogue,NaN,NaN,NaN
2,2,2,53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,Majora Carter,NaN,NaN,NaN
3,3,3,66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,Ken Robinson,NaN,NaN,NaN
4,4,4,92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",Hans Rosling,NaN,NaN,NaN


In [5]:
# Set the talk ID as the index and drop the unnecessary first two columns: 
ted_only = ted_only.set_index('Talk_ID')
ted_only = ted_only.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])

In [6]:
ted_only.head()

,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4
Talk_ID,,,,,,,,,,,,,
1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",Al Gore,NaN,NaN,NaN
7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",David Pogue,NaN,NaN,NaN
53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,Majora Carter,NaN,NaN,NaN
66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,Ken Robinson,NaN,NaN,NaN
92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",Hans Rosling,NaN,NaN,NaN


First, we want to know which talks only have one speaker. To do this, we only select the rows that `NaN` for the second speaker. 

In [51]:
talk1speak = ted_only[ted_only['speaker_2'].isnull()]

# https://stackoverflow.com/questions/43831539/how-to-select-rows-with-nan-in-particular-column

In [52]:
talk1speak.shape

(966, 13)

In [72]:
talk1 = talk1speak[["speaker_1","speaker_2","speaker_3","speaker_4"]]

In [74]:
talk1.head()


,speaker_1,speaker_2,speaker_3,speaker_4
Talk_ID,,,,
1,Al Gore,NaN,NaN,NaN
7,David Pogue,NaN,NaN,NaN
53,Majora Carter,NaN,NaN,NaN
66,Ken Robinson,NaN,NaN,NaN
92,Hans Rosling,NaN,NaN,NaN


We have 966 talks that have only one speaker. 


To figure out which talks have exactly two speakers, we select the rows that have a second speaker, but **not** a third one:  

In [64]:
s2temp = ted_only[ted_only['speaker_3'].isnull()]
talk2speak = s2temp[~s2temp['speaker_2'].isnull()]

In [65]:
talk2speak.shape

(25, 13)

In [75]:
talk2 = talk2speak[["speaker_1","speaker_2","speaker_3","speaker_4"]]

We have 25 talks that have two speakers. 


To figure out which talks have three speakers, we select the rows that have a third speaker, but **not** a fourth one:  

In [11]:
s3temp = ted_only[ted_only['speaker_4'].isnull()]
talk3speak = s3temp[~s3temp['speaker_3'].isnull()]

In [12]:
talk3speak.shape

(0, 13)

There are no talks with exactly three speakers. 


To figure out which talks have four speakers, we select the rows that have a fourth speaker:  

In [13]:
talk4speak = ted_only[~ted_only['speaker_4'].isnull()]


In [14]:
talk4speak.shape

(1, 13)

In [76]:
talk4 = talk4speak[["speaker_1","speaker_2","speaker_3","speaker_4"]]

As a quick check, we check if the total number of rows in the `talkNspeak` data frames equal the number of rows in `ted_only`:

In [15]:
ted_only.shape[0] == talk1speak.shape[0] + talk2speak.shape[0] + talk3speak.shape[0] + talk4speak.shape[0]

True

## Adding gender to the talks

Now that we have separate the talks into smaller data frames, each representing the number of speakers, we will be working to "gender" the talks. 

In [16]:
gender_slice = speakers[["speaker","Gender_handcheck"]]

In [17]:
gender_slice.head()

,speaker,Gender_handcheck
0,Al Gore,male
1,David Pogue,male
2,Majora Carter,female
3,Ken Robinson,male
4,Hans Rosling,male


In [27]:
gender_slice.set_index("speaker")

,Gender_handcheck
speaker,
Al Gore,male
David Pogue,male
Majora Carter,female
Ken Robinson,male
Hans Rosling,male
Tony Robbins,male
Joshua Prince-Ramus,male
Julia Sweeney,female
Rick Warren,male


### Gendering talks with one speaker

In [53]:
# We want to take speaker gender from speakers and put it into the talks
talk1speak = talk1speak.reset_index().merge(gender_slice, left_on = "speaker_1", right_on = "speaker", how = "left")

# We use `reset_index()` to preserve the `Talk_ID` column

In [54]:
talk1speak.head()

,Talk_ID,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4,speaker,Gender_handcheck
0,1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",Al Gore,NaN,NaN,NaN,Al Gore,male
1,7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",David Pogue,NaN,NaN,NaN,David Pogue,male
2,53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,Majora Carter,NaN,NaN,NaN,Majora Carter,female
3,66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,Ken Robinson,NaN,NaN,NaN,Ken Robinson,male
4,92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",Hans Rosling,NaN,NaN,NaN,Hans Rosling,male


In [55]:
# Reset the index back to `Talk_ID` 
talk1speak.set_index("Talk_ID", inplace = True)

In [56]:
talk1speak

,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4,speaker,Gender_handcheck
Talk_ID,,,,,,,,,,,,,,,
1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",Al Gore,NaN,NaN,NaN,Al Gore,male
7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",David Pogue,NaN,NaN,NaN,David Pogue,male
53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,Majora Carter,NaN,NaN,NaN,Majora Carter,female
66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,Ken Robinson,NaN,NaN,NaN,Ken Robinson,male
92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",Hans Rosling,NaN,NaN,NaN,Hans Rosling,male
96,https://www.ted.com/talks/tony_robbins_asks_wh...,Why we do what we do,"Tony Robbins discusses the ""invisible forces"" ...",TED2006,0:21:45,6/27/06,"entertainment,goal-setting,potential,psycholog...",22368699,Thank you. I have to tell you I'm both chall...,Tony Robbins,NaN,NaN,NaN,Tony Robbins,male
49,https://www.ted.com/talks/joshua_prince_ramus_...,Behind the design of Seattle's library,Architect Joshua Prince-Ramus takes the audien...,TED2006,0:19:58,7/10/06,"library,architecture,design,culture,collaboration",1042335,I'm going to present three projects in rapid...,Joshua Prince-Ramus,NaN,NaN,NaN,Joshua Prince-Ramus,male
86,https://www.ted.com/talks/julia_sweeney_on_let...,Letting go of God,When two young Mormon missionaries knock on Ju...,TED2006,0:16:32,7/10/06,"atheism,Christianity,religion,God,comedy,humor...",3903747,"On September 10, the morning of my seventh b...",Julia Sweeney,NaN,NaN,NaN,Julia Sweeney,female
71,https://www.ted.com/talks/rick_warren_on_a_lif...,A life of purpose,"Pastor Rick Warren, author of ""The Purpose-Dri...",TED2006,0:21:02,7/18/06,"Christianity,philanthropy,religion,God,happine...",3361934,"I'm often asked, ""What surprised you about t...",Rick Warren,NaN,NaN,NaN,Rick Warren,male


In [57]:
# Remove the second speaker column and rename the last column to "talk_gender"
talk1speak = talk1speak.drop(columns = ['speaker'])

In [58]:
talk1speak.head()

,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4,Gender_handcheck
Talk_ID,,,,,,,,,,,,,,
1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",Al Gore,NaN,NaN,NaN,male
7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",David Pogue,NaN,NaN,NaN,male
53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,Majora Carter,NaN,NaN,NaN,female
66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,Ken Robinson,NaN,NaN,NaN,male
92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",Hans Rosling,NaN,NaN,NaN,male


In [59]:
talk1speak.rename(columns={"Gender_handcheck": "talk_gender"}, inplace = True)

In [60]:
talk1speak.head()

,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4,talk_gender
Talk_ID,,,,,,,,,,,,,,
1,https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,With the same humor and humanity he exuded in ...,TED2006,0:16:17,6/27/06,"alternative energy,cars,global issues,climate ...",3266733,"Thank you so much, Chris. And it's truly a g...",Al Gore,NaN,NaN,NaN,male
7,https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,New York Times columnist David Pogue takes aim...,TED2006,0:21:26,6/27/06,"simplicity,entertainment,interface design,soft...",1702201,"(Music: ""The Sound of Silence,"" Simon & Garf...",David Pogue,NaN,NaN,NaN,male
53,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,"In an emotionally charged talk, MacArthur-winn...",TED2006,0:18:36,6/27/06,"MacArthur grant,cities,green,activism,politics...",2000421,If you're here today — and I'm very happy th...,Majora Carter,NaN,NaN,NaN,female
66,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,Sir Ken Robinson makes an entertaining and pro...,TED2006,0:19:24,6/27/06,"children,teaching,creativity,parenting,culture...",51614087,Good morning. How are you? (Laughter) ...,Ken Robinson,NaN,NaN,NaN,male
92,https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,You've never seen data presented like this. Wi...,TED2006,0:19:50,6/27/06,"demo,Asia,global issues,visualizations,global ...",12662135,"About 10 years ago, I took on the task to te...",Hans Rosling,NaN,NaN,NaN,male


In [21]:
talk1speak.shape

(966, 14)

### Gendering talks with one speaker

Now with that talks with just one speaker each have been "gendered," we turn our attention to the talks with multiple speakers. If the multiple speakers for a talk have the same gender, then we label the talk with that gender. 

We start with those that have two speakers. We will add the gender for the first speaker and then for the second speaker. Our procedure is similar to the above process for just one speaker. 

In [66]:
print(talk2speak.shape)

talk2speak.head()

(25, 13)


,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4
Talk_ID,,,,,,,,,,,,,
118,https://www.ted.com/talks/sergey_brin_and_larr...,The genesis of Google,Google co-founders Larry Page and Sergey Brin ...,TED2004,0:20:33,5/3/07,"web,design,Google,culture,business,technology,...",1529641,Sergey Brin: I want to discuss a question I ...,Sergey Brin,Larry Page,NaN,NaN
222,https://www.ted.com/talks/the_jill_and_julia_show,The Jill and Julia Show,"Two TED favorites, Jill Sobule and Julia Sween...",TED2007,0:06:14,2/20/08,"entertainment,comedy,humor,storytelling,collab...",507130,♫ Jill Sobule: At a conference in Monterey b...,Jill Sobule,Julia Sweeney,NaN,NaN
224,https://www.ted.com/talks/roy_gould_and_curtis...,A preview of the WorldWide Telescope,Educator Roy Gould and researcher Curtis Wong ...,TED2008,0:06:42,2/27/08,"telescopes,demo,astronomy,universe,science,tec...",1043036,"Roy Gould: Less than a year from now, the wo...",Roy Gould,Curtis Wong,NaN,NaN
246,https://www.ted.com/talks/tod_machover_and_dan...,Inventing instruments that unlock new music,Tod Machover of MIT's Media Lab is devoted to ...,TED2008,0:20:41,4/15/08,"demo,entertainment,writing,live music,health c...",519734,The first idea I'd like to suggest is that w...,Tod Machover,Dan Ellsey,NaN,NaN
322,https://www.ted.com/talks/bruno_bowden_folds_w...,Blindfold origami and cello,After Robert Lang's talk on origami at TED2008...,TED2008,0:02:58,8/1/08,"origami,entertainment,cello,music",384129,Hello everyone. And so the two of us are her...,Bruno Bowden,Rufus Cappadocia,NaN,NaN


We're going to make a copy of our talks with two speakers.

In [67]:
talk2temp = talk2speak.copy()

In [69]:
talk2temp.shape

(25, 13)

In [70]:
# Left merge to add the gender for the first speaker
talk2temp = talk2temp.reset_index().merge(gender_slice, left_on = "speaker_1", right_on = "speaker", how = "left")
talk2temp.set_index("Talk_ID", inplace = True)

# Drop the additional speaker column and rename the gender column 
talk2temp = talk2temp.drop(columns = ['speaker'])
talk2temp = talk2temp.rename(columns={"Gender_handcheck": "speaker_1_gender"})

talk2temp.head()

,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4,speaker_1_gender
Talk_ID,,,,,,,,,,,,,,
118,https://www.ted.com/talks/sergey_brin_and_larr...,The genesis of Google,Google co-founders Larry Page and Sergey Brin ...,TED2004,0:20:33,5/3/07,"web,design,Google,culture,business,technology,...",1529641,Sergey Brin: I want to discuss a question I ...,Sergey Brin,Larry Page,NaN,NaN,male
222,https://www.ted.com/talks/the_jill_and_julia_show,The Jill and Julia Show,"Two TED favorites, Jill Sobule and Julia Sween...",TED2007,0:06:14,2/20/08,"entertainment,comedy,humor,storytelling,collab...",507130,♫ Jill Sobule: At a conference in Monterey b...,Jill Sobule,Julia Sweeney,NaN,NaN,female
224,https://www.ted.com/talks/roy_gould_and_curtis...,A preview of the WorldWide Telescope,Educator Roy Gould and researcher Curtis Wong ...,TED2008,0:06:42,2/27/08,"telescopes,demo,astronomy,universe,science,tec...",1043036,"Roy Gould: Less than a year from now, the wo...",Roy Gould,Curtis Wong,NaN,NaN,male
246,https://www.ted.com/talks/tod_machover_and_dan...,Inventing instruments that unlock new music,Tod Machover of MIT's Media Lab is devoted to ...,TED2008,0:20:41,4/15/08,"demo,entertainment,writing,live music,health c...",519734,The first idea I'd like to suggest is that w...,Tod Machover,Dan Ellsey,NaN,NaN,male
322,https://www.ted.com/talks/bruno_bowden_folds_w...,Blindfold origami and cello,After Robert Lang's talk on origami at TED2008...,TED2008,0:02:58,8/1/08,"origami,entertainment,cello,music",384129,Hello everyone. And so the two of us are her...,Bruno Bowden,Rufus Cappadocia,NaN,NaN,male


In [71]:
# Repeat the above process on the second speaker
# Left merge to add the gender for the second speaker
talk2temp = talk2temp.reset_index().merge(gender_slice, left_on = "speaker_2", right_on = "speaker", how = "left")
talk2temp.set_index("Talk_ID", inplace = True)

# Drop the additional speaker column and rename the gender column 
talk2temp = talk2temp.drop(columns = ['speaker'])
talk2temp = talk2temp.rename(columns={"Gender_handcheck": "speaker_2_gender"})

talk2temp.head()

,public_url,headline,description,event,duration,published,tags,views,text,speaker_1,speaker_2,speaker_3,speaker_4,speaker_1_gender,speaker_2_gender
Talk_ID,,,,,,,,,,,,,,,
118,https://www.ted.com/talks/sergey_brin_and_larr...,The genesis of Google,Google co-founders Larry Page and Sergey Brin ...,TED2004,0:20:33,5/3/07,"web,design,Google,culture,business,technology,...",1529641,Sergey Brin: I want to discuss a question I ...,Sergey Brin,Larry Page,NaN,NaN,male,male
222,https://www.ted.com/talks/the_jill_and_julia_show,The Jill and Julia Show,"Two TED favorites, Jill Sobule and Julia Sween...",TED2007,0:06:14,2/20/08,"entertainment,comedy,humor,storytelling,collab...",507130,♫ Jill Sobule: At a conference in Monterey b...,Jill Sobule,Julia Sweeney,NaN,NaN,female,female
224,https://www.ted.com/talks/roy_gould_and_curtis...,A preview of the WorldWide Telescope,Educator Roy Gould and researcher Curtis Wong ...,TED2008,0:06:42,2/27/08,"telescopes,demo,astronomy,universe,science,tec...",1043036,"Roy Gould: Less than a year from now, the wo...",Roy Gould,Curtis Wong,NaN,NaN,male,male
246,https://www.ted.com/talks/tod_machover_and_dan...,Inventing instruments that unlock new music,Tod Machover of MIT's Media Lab is devoted to ...,TED2008,0:20:41,4/15/08,"demo,entertainment,writing,live music,health c...",519734,The first idea I'd like to suggest is that w...,Tod Machover,Dan Ellsey,NaN,NaN,male,male
322,https://www.ted.com/talks/bruno_bowden_folds_w...,Blindfold origami and cello,After Robert Lang's talk on origami at TED2008...,TED2008,0:02:58,8/1/08,"origami,entertainment,cello,music",384129,Hello everyone. And so the two of us are her...,Bruno Bowden,Rufus Cappadocia,NaN,NaN,male,male


Now we check which rows have the same genders for both speakers:

In [50]:
talk2speak[talk2speak["speaker_1_gender"] == talk2speak["speaker_2_gender"]].shape

(18, 15)

In [51]:
talk2speak[talk2speak["speaker_1_gender"] != talk2speak["speaker_2_gender"]].shape

(7, 15)